In [1]:
# 例: chunk_size=50 行ずつファイルを分割して定形プロンプトXと一緒に書き出す
chunk_size = 22
prompt_x = """

## 1. エスペラント語根リストについて
以下に示すのは、エスペラントの「語根」リストです（語根ではないものは末尾に # が付記されています）。  
各行には左から順に、  
1. エスペラント語根(ES)  
2. 信頼できる日本語訳(J)  
3. 信頼できる中国語訳(C1)(抜け落ち多数あり)  
4. 信頼できない中国語訳(C2)(google翻訳)  
5. 信頼できる英語訳(E)  
が「|」で区切られて示されています。

## 2. タスク概要
- 目的: 与えられたエスペラント語根全てに対して、リストの順に、簡体字中国語の漢字(基本的に1文字)を割り当てる。  
- ただし:
  - 1文字での割当が難しい、あるいは多画数・低頻度・汎用性が低いなど、学習コストが高すぎる場合は、2文字以上の熟語を割り当ててもよい。
  - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

## 3. エスペラント語根への漢字割当時に考慮してほしい4つの条件
1. **漢字・偏旁の種類の総数をできるだけ抑える**  
   - 漢字や偏旁の重複使用は大歓迎。学習者の負担を減らすためにも、可能な限り同じ漢字や偏旁を使い回してほしい。
   (同じ漢字については、後で上付き文字などによって区別する予定なので、現段階では一切気にせずどんどん同じ漢字や偏旁を使い回してほしい。)


2. **日本人・中国人双方が“意味を連想しやすい”簡体字であること**  
   - 簡体字を用いるが、日本人にとっても意味がつかみやすい漢字や熟語を優先して選んでほしい。

3. **できるだけ画数が少ない漢字を優先する**  
   - 非漢字圏の学習者の負担を減らすために、筆画の多い漢字より画数の少ない漢字を優先して割り当てたい。

4. **1文字割当が困難なら、既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討し、それでも難しければ割当を行わない**  
   - まず1文字で割当できないかを検討し、それが困難な場合は他のエスペラント語根に対して割り当てられている(もしくは別の回も含む他の箇所での割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討する。  
   - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

5. **多画数・低頻度・汎用性の低い漢字1文字を強引に割り当てるより、少画数・高頻度・汎用性の高い漢字のみを組み合わせた熟語を優先する**

   - たとえ漢字1文字で意味を表せても、その漢字の学習コストが高すぎる場合は2文字以上の熟語の方が望ましい。
   - 割当後の漢字・偏旁の種類の総数を抑えるためにも、なるべく多くの語根に転用しやすい（汎用性の高い）漢字や偏旁の組み合わせを優先する。

## 4. 作業の進め方・注意点
- 慎重かつ丁寧に、一切ミスがないよう検討してほしい。  
- リストの順に、エスペラント語根への割当を進めてほしい。


## 5. 各エスペラント語根への対応漢字を提案する際の推奨フォーマット  
**各エスペラント語根につき、以下の4項目(3と4は可能な範囲で)をスラッシュ「/」区切りで提示してください(スプレッドシート上での列分割を容易にするため)。1行につき1語根の対応を示す形式です。**

1. **エスペラント語根**  
2. **提案する簡体字(または熟語)**  
3. **(可能であれば)その漢字や熟語を選んだ理由や根拠**  
4. **(可能であれば)既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁との関係、学習コストに関する考察**  

### 5.1 出力例(イメージ)  
```
エスペラント語根1/提案漢字/選定理由/学習コストに関する考察
エスペラント語根2/提案熟語/選定理由/学習コストに関する考察
...
```

## 6. まとめ・最終要請
以上の指示に基づき、以下のエスペラント語根(各回約22件)全てに対する簡体字中国語の漢字や熟語の割当を、リストの順に、誤りがないよう慎重に丁寧に進めてください。  
学習コスト(漢字・偏旁の種類の総数)を極力抑えつつ、かつ日本語圏・中国語圏いずれの話者も概ね納得できる形での割当を、丁寧に提案・検討してください。  

どうぞよろしくお願いします。

# エスペラント語根リスト⇓
"""

input_file = "基于世界语词根的平均语义向量进行层次聚类排序后的,用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv"  # あなたのCSVファイルのパスを指定
output_file_base = "output_chunk_22_"  # 出力ファイル名のベース

# CSVファイルをリストに読み込む
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 全行数を把握
total_lines = len(lines)

# 50行ずつ切り出してファイル書き出し
# iは0, 50, 100, 150, ... のように進み、最後は4736を超える手前で止まる
for i in range(0, total_lines, chunk_size):
    # [i:i+chunk_size] で該当の50行を抜き出し
    chunk = lines[i:i + chunk_size]
    # どのチャンクかを数えやすいようにインデックスを +1 しておく
    chunk_index = i // chunk_size + 1  # 1,2,3,... のように連番が入る

    # 出力先ファイル名: "output_chunk_1.txt" のようになる
    output_file = f"{output_file_base}{chunk_index}.txt"

    # テキストファイルに書き出し
    with open(output_file, "w", encoding="utf-8") as out_f:
        # まず定形プロンプトXを書き出す
        out_f.write(prompt_x)
        out_f.write("\n")  # 改行(必要に応じて)
        # 続いて50行分のCSVの中身を書き出す
        for line in chunk:
            # 末尾の改行が残っているはずなのでそのまま書き出してもOK
            # もしstripしたい場合は out_f.write(line.strip() + "\n") などとする
            out_f.write(line)

    print(f"{output_file} を出力しました。")

print("すべての分割ファイルの出力が完了しました。")


output_chunk_22_1.txt を出力しました。
output_chunk_22_2.txt を出力しました。
output_chunk_22_3.txt を出力しました。
output_chunk_22_4.txt を出力しました。
output_chunk_22_5.txt を出力しました。
output_chunk_22_6.txt を出力しました。
output_chunk_22_7.txt を出力しました。
output_chunk_22_8.txt を出力しました。
output_chunk_22_9.txt を出力しました。
output_chunk_22_10.txt を出力しました。
output_chunk_22_11.txt を出力しました。
output_chunk_22_12.txt を出力しました。
output_chunk_22_13.txt を出力しました。
output_chunk_22_14.txt を出力しました。
output_chunk_22_15.txt を出力しました。
output_chunk_22_16.txt を出力しました。
output_chunk_22_17.txt を出力しました。
output_chunk_22_18.txt を出力しました。
output_chunk_22_19.txt を出力しました。
output_chunk_22_20.txt を出力しました。
output_chunk_22_21.txt を出力しました。
output_chunk_22_22.txt を出力しました。
output_chunk_22_23.txt を出力しました。
output_chunk_22_24.txt を出力しました。
output_chunk_22_25.txt を出力しました。
output_chunk_22_26.txt を出力しました。
output_chunk_22_27.txt を出力しました。
output_chunk_22_28.txt を出力しました。
output_chunk_22_29.txt を出力しました。
output_chunk_22_30.txt を出力しました。
output_chunk_22_31.txt を出力しました。
output_chunk_22_3

In [2]:
# 例: chunk_size=50 行ずつファイルを分割して定形プロンプトXと一緒に書き出す
chunk_size = 27
prompt_x = """

## 1. エスペラント語根リストについて
以下に示すのは、エスペラントの「語根」リストです（語根ではないものは末尾に # が付記されています）。  
各行には左から順に、  
1. エスペラント語根(ES)  
2. 信頼できる日本語訳(J)  
3. 信頼できる中国語訳(C1)(抜け落ち多数あり)  
4. 信頼できない中国語訳(C2)(google翻訳)  
5. 信頼できる英語訳(E)  
が「|」で区切られて示されています。

## 2. タスク概要
- 目的: 与えられたエスペラント語根全てに対して、リストの順に、簡体字中国語の漢字(基本的に1文字)を割り当てる。  
- ただし:
  - 1文字での割当が難しい、あるいは多画数・低頻度・汎用性が低いなど、学習コストが高すぎる場合は、2文字以上の熟語を割り当ててもよい。
  - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

## 3. エスペラント語根への漢字割当時に考慮してほしい4つの条件
1. **漢字・偏旁の種類の総数をできるだけ抑える**  
   - 漢字や偏旁の重複使用は大歓迎。学習者の負担を減らすためにも、可能な限り同じ漢字や偏旁を使い回してほしい。
   (同じ漢字については、後で上付き文字などによって区別する予定なので、現段階では一切気にせずどんどん同じ漢字や偏旁を使い回してほしい。)


2. **日本人・中国人双方が“意味を連想しやすい”簡体字であること**  
   - 簡体字を用いるが、日本人にとっても意味がつかみやすい漢字や熟語を優先して選んでほしい。

3. **できるだけ画数が少ない漢字を優先する**  
   - 非漢字圏の学習者の負担を減らすために、筆画の多い漢字より画数の少ない漢字を優先して割り当てたい。

4. **1文字割当が困難なら、既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討し、それでも難しければ割当を行わない**  
   - まず1文字で割当できないかを検討し、それが困難な場合は他のエスペラント語根に対して割り当てられている(もしくは別の回も含む他の箇所での割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討する。  
   - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

5. **多画数・低頻度・汎用性の低い漢字1文字を強引に割り当てるより、少画数・高頻度・汎用性の高い漢字のみを組み合わせた熟語を優先する**

   - たとえ漢字1文字で意味を表せても、その漢字の学習コストが高すぎる場合は2文字以上の熟語の方が望ましい。
   - 割当後の漢字・偏旁の種類の総数を抑えるためにも、なるべく多くの語根に転用しやすい（汎用性の高い）漢字や偏旁の組み合わせを優先する。

## 4. 作業の進め方・注意点
- 慎重かつ丁寧に、一切ミスがないよう検討してほしい。  
- リストの順に、エスペラント語根への割当を進めてほしい。


## 5. 各エスペラント語根への対応漢字を提案する際の推奨フォーマット  
**各エスペラント語根につき、以下の4項目(3と4は可能な範囲で)をスラッシュ「/」区切りで提示してください(スプレッドシート上での列分割を容易にするため)。1行につき1語根の対応を示す形式です。**

1. **エスペラント語根**  
2. **提案する簡体字(または熟語)**  
3. **(可能であれば)その漢字や熟語を選んだ理由や根拠**  
4. **(可能であれば)既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁との関係、学習コストに関する考察**  

### 5.1 出力例(イメージ)  
```
エスペラント語根1/提案漢字/選定理由/学習コストに関する考察
エスペラント語根2/提案熟語/選定理由/学習コストに関する考察
...
```

## 6. まとめ・最終要請
以上の指示に基づき、以下のエスペラント語根(各回約27件)全てに対する簡体字中国語の漢字や熟語の割当を、リストの順に、誤りがないよう慎重に丁寧に進めてください。  
学習コスト(漢字・偏旁の種類の総数)を極力抑えつつ、かつ日本語圏・中国語圏いずれの話者も概ね納得できる形での割当を、丁寧に提案・検討してください。  

どうぞよろしくお願いします。

# エスペラント語根リスト⇓
"""

input_file = "基于世界语词根的平均语义向量进行层次聚类排序后的,用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv"  # あなたのCSVファイルのパスを指定
output_file_base = "output_chunk_27_"  # 出力ファイル名のベース

# CSVファイルをリストに読み込む
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 全行数を把握
total_lines = len(lines)

# 50行ずつ切り出してファイル書き出し
# iは0, 50, 100, 150, ... のように進み、最後は4736を超える手前で止まる
for i in range(0, total_lines, chunk_size):
    # [i:i+chunk_size] で該当の50行を抜き出し
    chunk = lines[i:i + chunk_size]
    # どのチャンクかを数えやすいようにインデックスを +1 しておく
    chunk_index = i // chunk_size + 1  # 1,2,3,... のように連番が入る

    # 出力先ファイル名: "output_chunk_1.txt" のようになる
    output_file = f"{output_file_base}{chunk_index}.txt"

    # テキストファイルに書き出し
    with open(output_file, "w", encoding="utf-8") as out_f:
        # まず定形プロンプトXを書き出す
        out_f.write(prompt_x)
        out_f.write("\n")  # 改行(必要に応じて)
        # 続いて50行分のCSVの中身を書き出す
        for line in chunk:
            # 末尾の改行が残っているはずなのでそのまま書き出してもOK
            # もしstripしたい場合は out_f.write(line.strip() + "\n") などとする
            out_f.write(line)

    print(f"{output_file} を出力しました。")

print("すべての分割ファイルの出力が完了しました。")


output_chunk_27_1.txt を出力しました。
output_chunk_27_2.txt を出力しました。
output_chunk_27_3.txt を出力しました。
output_chunk_27_4.txt を出力しました。
output_chunk_27_5.txt を出力しました。
output_chunk_27_6.txt を出力しました。
output_chunk_27_7.txt を出力しました。
output_chunk_27_8.txt を出力しました。
output_chunk_27_9.txt を出力しました。
output_chunk_27_10.txt を出力しました。
output_chunk_27_11.txt を出力しました。
output_chunk_27_12.txt を出力しました。
output_chunk_27_13.txt を出力しました。
output_chunk_27_14.txt を出力しました。
output_chunk_27_15.txt を出力しました。
output_chunk_27_16.txt を出力しました。
output_chunk_27_17.txt を出力しました。
output_chunk_27_18.txt を出力しました。
output_chunk_27_19.txt を出力しました。
output_chunk_27_20.txt を出力しました。
output_chunk_27_21.txt を出力しました。
output_chunk_27_22.txt を出力しました。
output_chunk_27_23.txt を出力しました。
output_chunk_27_24.txt を出力しました。
output_chunk_27_25.txt を出力しました。
output_chunk_27_26.txt を出力しました。
output_chunk_27_27.txt を出力しました。
output_chunk_27_28.txt を出力しました。
output_chunk_27_29.txt を出力しました。
output_chunk_27_30.txt を出力しました。
output_chunk_27_31.txt を出力しました。
output_chunk_27_3

In [3]:
# 例: chunk_size=50 行ずつファイルを分割して定形プロンプトXと一緒に書き出す
chunk_size = 25
prompt_x = """

## 1. エスペラント語根リストについて
以下に示すのは、エスペラントの「語根」リストです（語根ではないものは末尾に # が付記されています）。  
各行には左から順に、  
1. エスペラント語根(ES)  
2. 信頼できる日本語訳(J)  
3. 信頼できる中国語訳(C1)(抜け落ち多数あり)  
4. 信頼できない中国語訳(C2)(google翻訳)  
5. 信頼できる英語訳(E)  
が「|」で区切られて示されています。

## 2. タスク概要
- 目的: 与えられたエスペラント語根全てに対して、リストの順に、簡体字中国語の漢字(基本的に1文字)を割り当てる。  
- ただし:
  - 1文字での割当が難しい、あるいは多画数・低頻度・汎用性が低いなど、学習コストが高すぎる場合は、2文字以上の熟語を割り当ててもよい。
  - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

## 3. エスペラント語根への漢字割当時に考慮してほしい4つの条件
1. **漢字・偏旁の種類の総数をできるだけ抑える**  
   - 漢字や偏旁の重複使用は大歓迎。学習者の負担を減らすためにも、可能な限り同じ漢字や偏旁を使い回してほしい。
   (同じ漢字については、後で上付き文字などによって区別する予定なので、現段階では一切気にせずどんどん同じ漢字や偏旁を使い回してほしい。)


2. **日本人・中国人双方が“意味を連想しやすい”簡体字であること**  
   - 簡体字を用いるが、日本人にとっても意味がつかみやすい漢字や熟語を優先して選んでほしい。

3. **できるだけ画数が少ない漢字を優先する**  
   - 非漢字圏の学習者の負担を減らすために、筆画の多い漢字より画数の少ない漢字を優先して割り当てたい。

4. **1文字割当が困難なら、既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討し、それでも難しければ割当を行わない**  
   - まず1文字で割当できないかを検討し、それが困難な場合は他のエスペラント語根に対して割り当てられている(もしくは別の回も含む他の箇所での割当に用いられるであろう、汎用性が高い)漢字や偏旁を組み合わせた熟語を検討する。  
   - それでもなお困難な場合、当該エスペラント語根への漢字割当は「未対応」のままとしても構わない。

5. **多画数・低頻度・汎用性の低い漢字1文字を強引に割り当てるより、少画数・高頻度・汎用性の高い漢字のみを組み合わせた熟語を優先する**

   - たとえ漢字1文字で意味を表せても、その漢字の学習コストが高すぎる場合は2文字以上の熟語の方が望ましい。
   - 割当後の漢字・偏旁の種類の総数を抑えるためにも、なるべく多くの語根に転用しやすい（汎用性の高い）漢字や偏旁の組み合わせを優先する。

## 4. 作業の進め方・注意点
- 慎重かつ丁寧に、一切ミスがないよう検討してほしい。  
- リストの順に、エスペラント語根への割当を進めてほしい。


## 5. 各エスペラント語根への対応漢字を提案する際の推奨フォーマット  
**各エスペラント語根につき、以下の4項目(3と4は可能な範囲で)をスラッシュ「/」区切りで提示してください(スプレッドシート上での列分割を容易にするため)。1行につき1語根の対応を示す形式です。**

1. **エスペラント語根**  
2. **提案する簡体字(または熟語)**  
3. **(可能であれば)その漢字や熟語を選んだ理由や根拠**  
4. **(可能であれば)既出の(または今後割当に用いられるであろう、汎用性が高い)漢字や偏旁との関係、学習コストに関する考察**  

### 5.1 出力例(イメージ)  
```
エスペラント語根1/提案漢字/選定理由/学習コストに関する考察
エスペラント語根2/提案熟語/選定理由/学習コストに関する考察
...
```

## 6. まとめ・最終要請
以上の指示に基づき、以下のエスペラント語根(各回約25件)全てに対する簡体字中国語の漢字や熟語の割当を、リストの順に、誤りがないよう慎重に丁寧に進めてください。  
学習コスト(漢字・偏旁の種類の総数)を極力抑えつつ、かつ日本語圏・中国語圏いずれの話者も概ね納得できる形での割当を、丁寧に提案・検討してください。  

どうぞよろしくお願いします。

# エスペラント語根リスト⇓
"""

input_file = "基于世界语词根的平均语义向量进行层次聚类排序后的,用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv"  # あなたのCSVファイルのパスを指定
output_file_base = "output_chunk_25_"  # 出力ファイル名のベース

# CSVファイルをリストに読み込む
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 全行数を把握
total_lines = len(lines)

# 50行ずつ切り出してファイル書き出し
# iは0, 50, 100, 150, ... のように進み、最後は4736を超える手前で止まる
for i in range(0, total_lines, chunk_size):
    # [i:i+chunk_size] で該当の50行を抜き出し
    chunk = lines[i:i + chunk_size]
    # どのチャンクかを数えやすいようにインデックスを +1 しておく
    chunk_index = i // chunk_size + 1  # 1,2,3,... のように連番が入る

    # 出力先ファイル名: "output_chunk_1.txt" のようになる
    output_file = f"{output_file_base}{chunk_index}.txt"

    # テキストファイルに書き出し
    with open(output_file, "w", encoding="utf-8") as out_f:
        # まず定形プロンプトXを書き出す
        out_f.write(prompt_x)
        out_f.write("\n")  # 改行(必要に応じて)
        # 続いて50行分のCSVの中身を書き出す
        for line in chunk:
            # 末尾の改行が残っているはずなのでそのまま書き出してもOK
            # もしstripしたい場合は out_f.write(line.strip() + "\n") などとする
            out_f.write(line)

    print(f"{output_file} を出力しました。")

print("すべての分割ファイルの出力が完了しました。")


output_chunk_25_1.txt を出力しました。
output_chunk_25_2.txt を出力しました。
output_chunk_25_3.txt を出力しました。
output_chunk_25_4.txt を出力しました。
output_chunk_25_5.txt を出力しました。
output_chunk_25_6.txt を出力しました。
output_chunk_25_7.txt を出力しました。
output_chunk_25_8.txt を出力しました。
output_chunk_25_9.txt を出力しました。
output_chunk_25_10.txt を出力しました。
output_chunk_25_11.txt を出力しました。
output_chunk_25_12.txt を出力しました。
output_chunk_25_13.txt を出力しました。
output_chunk_25_14.txt を出力しました。
output_chunk_25_15.txt を出力しました。
output_chunk_25_16.txt を出力しました。
output_chunk_25_17.txt を出力しました。
output_chunk_25_18.txt を出力しました。
output_chunk_25_19.txt を出力しました。
output_chunk_25_20.txt を出力しました。
output_chunk_25_21.txt を出力しました。
output_chunk_25_22.txt を出力しました。
output_chunk_25_23.txt を出力しました。
output_chunk_25_24.txt を出力しました。
output_chunk_25_25.txt を出力しました。
output_chunk_25_26.txt を出力しました。
output_chunk_25_27.txt を出力しました。
output_chunk_25_28.txt を出力しました。
output_chunk_25_29.txt を出力しました。
output_chunk_25_30.txt を出力しました。
output_chunk_25_31.txt を出力しました。
output_chunk_25_3